In [1]:
# [기본 세팅] ------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from IPython.display import display

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# 윈도우 한글 폰트 + 마이너스 표시 깨짐 방지
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 표시에 도움되는 옵션 (너무 길게 줄바꿈되는 것 방지)
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

print("✅ 기본 세팅 완료")


✅ 기본 세팅 완료


In [2]:
# [경로 유틸] ------------------------------------------------
import os

def _path(*candidates):
    """
    여러 경로 후보 중 '존재하는' 첫 번째 경로를 반환.
    예: _path("./a.csv", "/mnt/data/a.csv")
    """
    for p in candidates:
        if p and os.path.exists(p):
            return p
    return None  # 없으면 None

# [파일 경로 확인] --------------------------------------------
FN = {
    "orders": "olist_orders_dataset.csv",
    "order_items": "olist_order_items_dataset.csv",
    "order_payments": "olist_order_payments_dataset.csv",
    "order_reviews": "olist_order_reviews_dataset.csv",
    "customers": "olist_customers_dataset.csv",
    "geolocation": "olist_geolocation_dataset.csv",
    "products": "olist_products_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "category_tr": "product_category_name_translation.csv",
}

def resolve_path(filename):
    """
    현재 폴더(./) 우선, 없으면 /mnt/data 에서 찾기.
    둘 다 없으면 FileNotFoundError.
    """
    here = os.path.join(".", filename)
    data = os.path.join("/mnt/data", filename)
    p = _path(here, data)
    if p is None:
        raise FileNotFoundError(f"파일이 보이지 않습니다: {filename} (현재폴더 또는 /mnt/data)")
    return p


In [3]:
# [로더] ------------------------------------------------------
def load_all():
    """
    Olist 9개 CSV를 읽어 DataFrame dict로 반환.
    - 날짜 컬럼은 parse_dates로 바로 datetime 변환
    - ID/ZIP 등은 문자열(str)로 유지 (앞자리 0 보존)
    - 금액/회차 등은 숫자형으로 안전 변환(to_numeric)
    """
    # 1) 각 CSV 로드 (날짜 컬럼은 parse_dates 지정)
    orders = pd.read_csv(
        resolve_path(FN["orders"]),
        parse_dates=[
            "order_purchase_timestamp",
            "order_approved_at",
            "order_delivered_carrier_date",
            "order_delivered_customer_date",
            "order_estimated_delivery_date",
        ],
        low_memory=False,
    )
    order_items = pd.read_csv(
        resolve_path(FN["order_items"]),
        parse_dates=["shipping_limit_date"],
        low_memory=False,
    )
    order_payments = pd.read_csv(resolve_path(FN["order_payments"]), low_memory=False)
    order_reviews = pd.read_csv(
        resolve_path(FN["order_reviews"]),
        parse_dates=["review_creation_date", "review_answer_timestamp"],
        low_memory=False,
    )
    customers = pd.read_csv(resolve_path(FN["customers"]), low_memory=False)
    geolocation = pd.read_csv(resolve_path(FN["geolocation"]), low_memory=False)
    products = pd.read_csv(resolve_path(FN["products"]), low_memory=False)
    sellers = pd.read_csv(resolve_path(FN["sellers"]), low_memory=False)
    category_tr = pd.read_csv(resolve_path(FN["category_tr"]), low_memory=False)

    # 2) 문자열로 유지할 컬럼들 통일
    def _as_str(df, cols):
        for c in cols:
            if c in df.columns:
                df[c] = df[c].astype(str)
        return df

    orders = _as_str(orders, ["order_id", "customer_id"])
    order_items = _as_str(order_items, ["order_id", "product_id", "seller_id"])
    order_payments = _as_str(order_payments, ["order_id", "payment_type"])
    order_reviews = _as_str(order_reviews, ["review_id", "order_id"])
    customers = _as_str(customers, ["customer_id", "customer_unique_id", "customer_zip_code_prefix"])
    geolocation = _as_str(geolocation, ["geolocation_zip_code_prefix"])
    products = _as_str(products, ["product_id", "product_category_name"])
    sellers = _as_str(sellers, ["seller_id", "seller_zip_code_prefix"])
    category_tr = _as_str(category_tr, ["product_category_name", "product_category_name_english"])

    # 3) 숫자형 컬럼 안전 변환 (문자 섞였을 경우를 대비)
    if "price" in order_items.columns:
        order_items["price"] = pd.to_numeric(order_items["price"], errors="coerce")
    if "freight_value" in order_items.columns:
        order_items["freight_value"] = pd.to_numeric(order_items["freight_value"], errors="coerce")
    if "payment_value" in order_payments.columns:
        order_payments["payment_value"] = pd.to_numeric(order_payments["payment_value"], errors="coerce")
    if "payment_installments" in order_payments.columns:
        order_payments["payment_installments"] = pd.to_numeric(order_payments["payment_installments"], errors="coerce")

    # 4) dict로 묶어서 반환
    raw = {
        "orders": orders,
        "order_items": order_items,
        "order_payments": order_payments,
        "order_reviews": order_reviews,
        "customers": customers,
        "geolocation": geolocation,
        "products": products,
        "sellers": sellers,
        "category_tr": category_tr,
    }
    return raw

# 실제 로드
raw = load_all()
print("✅ CSV 로드 완료")


✅ CSV 로드 완료


In [4]:
# 작업용 복사본(DataFrame 메모리에서만 사용) -------------------
df_orders         = raw["orders"].copy()
df_order_items    = raw["order_items"].copy()
df_payments       = raw["order_payments"].copy()
df_reviews        = raw["order_reviews"].copy()
df_customers      = raw["customers"].copy()
df_geolocation    = raw["geolocation"].copy()
df_products       = raw["products"].copy()
df_sellers        = raw["sellers"].copy()
df_category_tr    = raw["category_tr"].copy()

print("✅ 작업용 복사본 준비 완료")

✅ 작업용 복사본 준비 완료


In [5]:
# [EDA 유틸] --------------------------------------------------
def quick_shape_info(dfs: dict, head_rows: int = 2):
    """
    각 DF의 shape와 head 미리보기.
    파악이 목적이므로 출력 위주(저장 없음).
    """
    rows = []
    for name, d in dfs.items():
        rows.append((name, d.shape[0], d.shape[1]))
    info = pd.DataFrame(rows, columns=["name", "n_rows", "n_cols"]).sort_values("name")
    print("=== Shapes ===")
    display(info)

    for name, d in dfs.items():
        print(f"\n--- {name} : head({head_rows}) ---")
        display(d.head(head_rows))

def missing_report(df: pd.DataFrame):
    """
    단일 DF의 결측치 리포트 (개수/비율).
    """
    m = df.isna().sum().to_frame("missing")
    m["missing_pct"] = (m["missing"] / len(df) * 100).round(2)
    return m.sort_values("missing", ascending=False)

def missing_report_all(dfs: dict):
    """
    모든 DF에 같은 결측 리포트를 적용해 dict로 반환.
    """
    out = {}
    for name, d in dfs.items():
        out[name] = missing_report(d)
    return out

def numeric_summary(df: pd.DataFrame):
    """
    숫자형 컬럼 기술통계(사분위/IQR 포함).
    """
    num = df.select_dtypes(include=[np.number])
    if num.empty:
        return pd.DataFrame()
    desc = num.describe(percentiles=[.25, .5, .75]).T
    desc["iqr"] = desc["75%"] - desc["25%"]
    return desc

def iqr_outlier_mask(series: pd.Series, k: float = 1.5):
    """
    IQR 방식 이상치 마스크(True=이상치).
    제거/대체는 이 마스크를 이용해 사용자가 결정.
    """
    x = pd.to_numeric(series, errors="coerce")
    q1, q3 = x.quantile(0.25), x.quantile(0.75)
    iqr = q3 - q1
    lo, hi = q1 - k*iqr, q3 + k*iqr
    return (x < lo) | (x > hi)

# [조인 키 안내] ----------------------------------------------
# - orders (order_id, customer_id)
# - customers (customer_id -> customer_unique_id, state/city/zip)
# - order_items (order_id -> product_id, seller_id, price, freight_value)
# - order_payments (order_id -> payment_value, type)
# - order_reviews (order_id -> review_score, review_creation_date, ...)
# - products (product_id -> product_category_name)
# - category_tr (product_category_name -> product_category_name_english)
# - sellers (seller_id -> seller_city/state/zip)
# - geolocation (zip prefix 기준 공간 정보 - 직접 조인보다는 보조 참조)


In [6]:
# [첫 EDA 실행 예] --------------------------------------------
dfs_now = {
    "orders": df_orders, "order_items": df_order_items, "order_payments": df_payments,
    "order_reviews": df_reviews, "customers": df_customers, "geolocation": df_geolocation,
    "products": df_products, "sellers": df_sellers, "category_tr": df_category_tr,
}

# 1) 전체 크기와 미리보기
quick_shape_info(dfs_now, head_rows=2)

# 2) 결측치 요약 (각 DF 상위 10행만 출력)
print("\n=== 결측치 요약(각 DF 상위 10열) ===")
# miss_all = {k: v.head(10) for k, v in missing_report_all(dfs_now).items()}
miss_all = {}  # 빈 dict 준비
for k, v in missing_report_all(dfs_now).items():  # k=이름, v=결측리포트 DF
    top10 = v.head(10)  # 상위 10행만 미리보기
    miss_all[k] = top10  # 새 딕셔너리에 저장

# miss_all["orders"] 처럼 접근하면 상위 10행 리포트를 볼 수 있음

for k, v in miss_all.items():
    print(f"\n[{k}]")
    display(v)

# 3) 숫자형 요약 예시 (order_items 기준)
print("\n=== 숫자 요약 예시: order_items ===")
num_summary = numeric_summary(df_order_items)[["count","mean","std","min","25%","50%","75%","max","iqr"]]
display(num_summary.round(3).head(10))

print("\n✅ 초기화 완료: raw_* → df_* 복사 생성, EDA 유틸 준비 (저장 없음)")


=== Shapes ===


,name,n_rows,n_cols
8,category_tr,71,2
4,customers,99441,5
5,geolocation,1000163,5
1,order_items,112650,7
2,order_payments,103886,5
3,order_reviews,99224,7
0,orders,99441,8
6,products,32951,9
7,sellers,3095,4



--- orders : head(2) ---


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13



--- order_items : head(2) ---


...


[category_tr]


,missing,missing_pct
product_category_name,0,0.0
product_category_name_english,0,0.0



=== 숫자 요약 예시: order_items ===


,count,mean,std,min,25%,50%,75%,max,iqr
order_item_id,112650.0,1.198,0.705,1.00,1.00,1.00,1.00,21.00,0.00
price,112650.0,120.654,183.634,0.85,39.90,74.99,134.90,6735.00,95.00
freight_value,112650.0,19.990,15.806,0.00,13.08,16.26,21.15,409.68,8.07



✅ 초기화 완료: raw_* → df_* 복사 생성, EDA 유틸 준비 (저장 없음)


In [7]:
# =========================================================
# (1) 파라미터
# - 전제: 이전에 'Clean Start' 셀에서 df_*들이 메모리에 있음
# - 목표: 이 셀에서 전역 파라미터(임계값)만 조정할 수 있게 함
# =========================================================

# 결측행 삭제 임계값(비율): 결측 비율이 작으면 drop, 크면 그룹/전체 중앙값 대체
DROP_MISSING_THRESH = 0.005   # 0.5%

# 윈저라이즈 경계(백분위)
WINSOR_LO = 0.01              # 하위 1%
WINSOR_HI = 0.99              # 상위 1%

print("✅ 파라미터 설정 완료")

✅ 파라미터 설정 완료


In [8]:
# =========================================================
# (2) 유틸 함수
# - 여기 함수들은 이후 블럭에서 공통으로 사용
# =========================================================

def pct_missing(s: pd.Series) -> float:
    """시리즈의 결측 비율(0~1)을 반환."""
    s = pd.to_numeric(s, errors="ignore")
    return float(s.isna().mean())

def winsorize(s: pd.Series, lo=0.01, hi=0.99):
    """
    윈저라이즈: 극단값을 경계값으로 잘라내기(clip).
    반환: (윈저 적용 시리즈, 하한, 상한)
    """
    x = pd.to_numeric(s, errors="coerce")
    ql, qh = x.quantile(lo), x.quantile(hi)
    return x.clip(ql, qh), float(ql), float(qh)

def impute_with_group_median(df, target_col, group_col, global_median=None):
    """
    그룹 중앙값으로 결측을 채우고, 그래도 남으면 전체 중앙값으로 채우기.
    - target_col: 채울 대상 컬럼
    - group_col : 그룹 기준 컬럼 (예: product_id, order_id)
    """
    x = df[target_col]
    if global_median is None:
        global_median = pd.to_numeric(x, errors="coerce").median()
    grp_med = (df.groupby(group_col)[target_col]
                 .median(numeric_only=True))
    df[target_col] = df[target_col].fillna(df[group_col].map(grp_med))
    df[target_col] = df[target_col].fillna(global_median)
    return df

def count_nonpositive(s: pd.Series) -> int:
    """0 이하(비양수) 값 개수를 세기."""
    x = pd.to_numeric(s, errors="coerce")
    return int((x <= 0).sum())


In [9]:
# =========================================================
# (3) REVIEWS: 점수 정리 + 주문별 리뷰 유무 플래그 생성
# - review_score는 1~5만 유효 → 범위 밖/비수치 NaN
# - 주문별(review_count>0) → has_review=True
# - orders에 has_review 병합
# =========================================================

# 원본 복사
df_reviews_clean = df_reviews.copy()
df_orders_clean  = df_orders.copy()

# 1) 리뷰 점수 정리(1~5만 유효)
if "review_score" in df_reviews_clean.columns:
    df_reviews_clean["review_score"] = pd.to_numeric(df_reviews_clean["review_score"], errors="coerce")
    bad = ~df_reviews_clean["review_score"].isin([1,2,3,4,5])
    df_reviews_clean.loc[bad, "review_score"] = np.nan

# 2) 주문별 리뷰 유무
has_review_by_order = (
    df_reviews_clean.groupby("order_id")["review_id"]
                    .nunique()
                    .rename("review_count")
                    .reset_index()
)
has_review_by_order["has_review"] = has_review_by_order["review_count"] > 0

# 3) orders에 병합(없으면 False)
df_orders_clean = df_orders_clean.merge(
    has_review_by_order[["order_id","has_review"]],
    on="order_id", how="left"
)
df_orders_clean["has_review"] = df_orders_clean["has_review"].fillna(False)

print("✅ 리뷰 정리 + 주문 플래그 병합 완료")


✅ 리뷰 정리 + 주문 플래그 병합 완료


C:\Users\배성우\AppData\Local\Temp\ipykernel_27564\1515754481.py:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_orders_clean["has_review"] = df_orders_clean["has_review"].fillna(False)


In [10]:
# =========================================================
# (4) ORDER_ITEMS: price / freight_value 처리
# - 결측 비율 낮음: 행 drop
# - 결측 비율 큼  : product_id 그룹 중앙값 → 남으면 전체 중앙값
# - 음수는 NaN으로 바꾸고 위 로직 재사용
# - 윈저라이즈 후 *_wz 컬럼으로 보관
# =========================================================

df_order_items_clean = df_order_items.copy()

for col in ["price", "freight_value"]:
    if col not in df_order_items_clean.columns:
        continue

    # 1) 결측 전략 선택
    miss_ratio = pct_missing(df_order_items_clean[col])
    if miss_ratio <= DROP_MISSING_THRESH:
        # 결측행 삭제
        df_order_items_clean = df_order_items_clean[~df_order_items_clean[col].isna()].copy()
        impute_method = "drop_rows"
    else:
        # product_id 그룹 중앙값 → 남으면 전체 중앙값
        df_order_items_clean = impute_with_group_median(
            df_order_items_clean, col, "product_id"
        )
        impute_method = "group_median"

    # 2) 음수 보정 → NaN 후 위 전략 반복
    neg_mask = pd.to_numeric(df_order_items_clean[col], errors="coerce") < 0
    if neg_mask.any():
        df_order_items_clean.loc[neg_mask, col] = np.nan
        # 다시 결측 처리
        if impute_method == "drop_rows" and pct_missing(df_order_items_clean[col]) <= DROP_MISSING_THRESH:
            df_order_items_clean = df_order_items_clean[~df_order_items_clean[col].isna()].copy()
        else:
            gmed = pd.to_numeric(df_order_items_clean[col], errors="coerce").median()
            df_order_items_clean[col] = df_order_items_clean[col].fillna(gmed)

    # 3) 윈저라이즈(결과를 새 컬럼 *_wz로 보관)
    wz, lo, hi = winsorize(df_order_items_clean[col], WINSOR_LO, WINSOR_HI)
    df_order_items_clean[f"{col}_wz"] = wz

print("✅ 주문 아이템(price/freight) 처리 완료")


✅ 주문 아이템(price/freight) 처리 완료


C:\Users\배성우\AppData\Local\Temp\ipykernel_27564\3019872474.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  s = pd.to_numeric(s, errors="ignore")
C:\Users\배성우\AppData\Local\Temp\ipykernel_27564\3019872474.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  s = pd.to_numeric(s, errors="ignore")


In [11]:
# =========================================================
# (5) PAYMENTS: payment_value 처리
# - 0 이하(비양수)는 NaN
# - 결측 비율 낮음: 행 drop
# - 결측 비율 큼  : order_id 그룹 중앙값 → 남으면 전체 중앙값
# - 윈저라이즈 후 'payment_value_wz'에 보관
# =========================================================

df_payments_clean = df_payments.copy()

if "payment_value" in df_payments_clean.columns:
    # 1) 비양수 → NaN
    pv = pd.to_numeric(df_payments_clean["payment_value"], errors="coerce")
    bad_mask = pv <= 0
    if bad_mask.any():
        df_payments_clean.loc[bad_mask, "payment_value"] = np.nan

    # 2) 결측 전략
    miss_ratio = pct_missing(df_payments_clean["payment_value"])
    if miss_ratio <= DROP_MISSING_THRESH:
        df_payments_clean = df_payments_clean[~df_payments_clean["payment_value"].isna()].copy()
    else:
        # order_id 그룹 중앙값
        med_by_order = (df_payments_clean.groupby("order_id")["payment_value"]
                        .median(numeric_only=True))
        df_payments_clean["payment_value"] = df_payments_clean["payment_value"].fillna(
            df_payments_clean["order_id"].map(med_by_order)
        )
        # 그래도 남으면 전체 중앙값
        gmed = pd.to_numeric(df_payments_clean["payment_value"], errors="coerce").median()
        df_payments_clean["payment_value"] = df_payments_clean["payment_value"].fillna(gmed)

    # 3) 윈저라이즈
    pv_wz, lo_pv, hi_pv = winsorize(df_payments_clean["payment_value"], WINSOR_LO, WINSOR_HI)
    df_payments_clean["payment_value_wz"] = pv_wz

print("✅ 결제 금액(payment_value) 처리 완료")


✅ 결제 금액(payment_value) 처리 완료


C:\Users\배성우\AppData\Local\Temp\ipykernel_27564\3019872474.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  s = pd.to_numeric(s, errors="ignore")


In [12]:
# =========================================================
# (6) PRODUCTS & CATEGORY: 영문 카테고리 매핑
# - products.product_category_name → category_tr 영어명
# - 매핑 실패/결측은 'unknown'
# =========================================================

df_products_clean     = df_products.copy()
df_category_tr_clean  = df_category_tr.copy()

if "product_category_name" in df_products_clean.columns:
    cat_map = (df_category_tr_clean
               .set_index("product_category_name")["product_category_name_english"]
               .to_dict())
    df_products_clean["category_en"] = df_products_clean["product_category_name"].map(cat_map)
    df_products_clean["category_en"] = df_products_clean["category_en"].fillna("unknown")

print("✅ 카테고리 영문 매핑 완료")


✅ 카테고리 영문 매핑 완료


In [13]:
# =========================================================
# (7) CUSTOMERS / GEO / SELLERS: 변경 없음 → 복사만
# =========================================================
df_customers_clean   = df_customers.copy()
df_geolocation_clean = df_geolocation.copy()
df_sellers_clean     = df_sellers.copy()

# 이후 분석에서 쓰기 편하도록 별칭(alias) 바인딩
orders_clean       = df_orders_clean
order_items_clean  = df_order_items_clean
payments_clean     = df_payments_clean
reviews_clean      = df_reviews_clean
customers_clean    = df_customers_clean
geolocation_clean  = df_geolocation_clean
products_clean     = df_products_clean
sellers_clean      = df_sellers_clean
category_tr_clean  = df_category_tr_clean

print("✅ 변경 없는 테이블 복사/바인딩 완료")


✅ 변경 없는 테이블 복사/바인딩 완료


In [14]:
# =========================================================
# (8) 처리 요약 리포트 출력 (clean_report)
# - 이후 단계(조인/KPI/시각화)로 넘어가기 전 상태 점검용
# =========================================================
clean_report = {}

# reviews
if "review_score" in df_reviews_clean.columns:
    clean_report["reviews"] = {
        "review_score_missing_pct": pct_missing(df_reviews_clean["review_score"]),
        "orders_with_review_pct": float(df_orders_clean["has_review"].mean())
    }

# order_items
for col in ["price", "freight_value"]:
    if col in df_order_items_clean.columns:
        key = f"order_items.{col}"
        clean_report[key] = {
            "missing_pct_after": pct_missing(df_order_items_clean[col]),
            "winsor_bounds": (
                float(df_order_items_clean[f"{col}_wz"].quantile(WINSOR_LO)),
                float(df_order_items_clean[f"{col}_wz"].quantile(WINSOR_HI)),
            ) if f"{col}_wz" in df_order_items_clean.columns else None
        }

# payments
if "payment_value" in df_payments_clean.columns:
    clean_report["payments.payment_value"] = {
        "missing_pct_after": pct_missing(df_payments_clean["payment_value"]),
        "nonpositive_after": count_nonpositive(df_payments_clean["payment_value"]),
        "winsor_bounds": (
            float(df_payments_clean["payment_value_wz"].quantile(WINSOR_LO)),
            float(df_payments_clean["payment_value_wz"].quantile(WINSOR_HI)),
        ) if "payment_value_wz" in df_payments_clean.columns else None
    }

# products
if "category_en" in df_products_clean.columns:
    clean_report["products.category_en"] = {
        "unknown_pct": float((df_products_clean["category_en"] == "unknown").mean())
    }

print("=== CLEAN REPORT (요약) ===")
for k, v in clean_report.items():
    print(f"- {k}: {v}")

print("\n✅ 결측치/이상치 처리 완료 (*_clean DataFrame 사용)")


C:\Users\배성우\AppData\Local\Temp\ipykernel_27564\3019872474.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  s = pd.to_numeric(s, errors="ignore")
C:\Users\배성우\AppData\Local\Temp\ipykernel_27564\3019872474.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  s = pd.to_numeric(s, errors="ignore")
C:\Users\배성우\AppData\Local\Temp\ipykernel_27564\3019872474.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  s = pd.to_numeric(s, errors="ignore")


=== CLEAN REPORT (요약) ===
- reviews: {'review_score_missing_pct': 0.0, 'orders_with_review_pct': 0.9922768274655324}
- order_items.price: {'missing_pct_after': 0.0, 'winsor_bounds': (9.99, 890.0)}
- order_items.freight_value: {'missing_pct_after': 0.0, 'winsor_bounds': (4.424798, 84.52)}
- payments.payment_value: {'missing_pct_after': 0.0, 'nonpositive_after': 0, 'winsor_bounds': (6.7588479999999995, 1039.9063360000005)}
- products.category_en: {'unknown_pct': 0.01890686170374192}

✅ 결측치/이상치 처리 완료 (*_clean DataFrame 사용)


In [15]:
# === 번들로 저장 (권장: Pickle) ===
# === 다른 탭에서 clean DF를 그대로 쓰기 위해서 저장 ===
import os, pickle

# 저장 폴더
SNAP_DIR = "_snapshots"
os.makedirs(SNAP_DIR, exist_ok=True)

# 번들 구성 (요청하신 9개 DF)
bundle = {
    "orders_clean": orders_clean,
    "order_items_clean": order_items_clean,
    "payments_clean": payments_clean,
    "reviews_clean": reviews_clean,
    "customers_clean": customers_clean,
    "geolocation_clean": geolocation_clean,
    "products_clean": products_clean,
    "sellers_clean": sellers_clean,
    "category_tr_clean": category_tr_clean,
}

# 저장
BUNDLE_PATH = os.path.join(SNAP_DIR, "olist_clean_bundle.pkl")
with open(BUNDLE_PATH, "wb") as f:
    pickle.dump(bundle, f, protocol=pickle.HIGHEST_PROTOCOL)

print("✅ Saved bundle ->", BUNDLE_PATH)
print({k: v.shape for k, v in bundle.items()})

✅ Saved bundle -> _snapshots\olist_clean_bundle.pkl
{'orders_clean': (99441, 9), 'order_items_clean': (112650, 9), 'payments_clean': (103877, 6), 'reviews_clean': (99224, 7), 'customers_clean': (99441, 5), 'geolocation_clean': (1000163, 5), 'products_clean': (32951, 10), 'sellers_clean': (3095, 4), 'category_tr_clean': (71, 2)}
